# EEL7065 - Projeto de filtro para detecção de tons DTMF
Projetos de filtros baseado no tutorial: [Designing an FIR filter in Python](http://www.ee.iitm.ac.in/~nitin/teaching/ee5480/firdesign.html)

Comentários no código em inglês para possibilitar reuso e publicação eventualmente.

In [ ]:
# Import functions to keep it simple to use
from pylab import * 
import matplotlib.pyplot as plt
import numpy as np
import wave
import sys
import simpleaudio as sa
from goertzel import goertzel

# charts styling
plt.style.use('ggplot')
#plt.style.use('seaborn-notebook')
from pylab import rcParams
#rcParams['figure.figsize'] = 13, 8 # increases figure size
#matplotlib.rcParams.update({'font.size': 14}) # increases chart font size
#rcParams['font.family'] = 'Arial Narrow'

## Parâmetros gerais

In [ ]:
#fs = 44100            # sampling frequency [Hz]
#Ts = 1.0/fs           # sampling period [s]
#noise_amp = 1         # noise amplitude 

# time steps: 1 second of data samples at spacing of 1/1000 seconds
#t = arange(0, 1, Ts)
#t = np.linspace(0, 1, fs, endpoint=False)


## Sinal de entrada
Exemplo de áudio gravado.

In [ ]:
#spf = wave.open('voice_example_mono.wav','r')
spf = wave.open('testall_mono.wav','r') # sample file provided

# extract raw audio from wav file
s = spf.readframes(-1)
s = np.fromstring(s, 'Int16')

# shortens audio for testing
s = s[0:int(round(len(s)/2))]

# extract signal properties
num_channels = spf.getnchannels()
bytes_per_sample = spf.getsampwidth()
sample_rate = spf.getframerate()
num_frames = spf.getnframes()
duration = num_frames / float(sample_rate) # gets duration in seconds


nyq_rate = sample_rate / 2.0

# if stereo, prints error
if num_channels == 2:
    print ('Just mono files accepted.')
    sys.exit(0)

print('Frequência de amostragem (Hz): ' + str(sample_rate))
print('Frequência de Nyquist    (Hz): ' + str(nyq_rate))
print('Bits por amostra             : ' + str(bytes_per_sample*8))
print('Duração                   (s): ' + str(duration))

#plt.figure(1)
plt.xlabel('Samples (n)')
plt.title('Voice signal')
plt.plot(s)
plt.show()

## Espectro do sinal de voz

In [ ]:
# gets fft frequency to plot properly
n = s.size
timestep = 1.0 / sample_rate
freq = np.fft.fftfreq(n, d=timestep)

#ft_pure = fft(s_pure)/len(s_pure)
ft_s = fft(s)/len(s)

#plt.plot(20*log10(abs(ft_pure)))
plt.plot(freq, 20*log10(abs(ft_s)))
plt.title('Voice signal')
plt.ylabel('Magnitude (dB)')
plt.xlabel('Frequency (Hz)')
#plt.legend(['voice signal wave FFT'], loc='best')
plt.show()

In [ ]:
# low and high frequencies from DTMF standard
#lf = np.array([697, 770, 852, 941])
#hf = np.array([1209, 1336, 1477, 1633])

# make matrix/data structure to hold combination of frequencies to be used later

# create input signals mixing the combinations
#s1 = cos(2*pi*697*t)
#s2 = cos(2*pi*1209*t)

#s3 = s1+s2

# test some filters to extract those

# yo


## Geração dos tons DTMF

## Visualização dos tons na frequência 

In [ ]:
# gets fft frequency to plot properly
n = tones.size
timestep = 1.0 / sample_rate
freq = np.fft.fftfreq(n, d=timestep)

#ft_pure = fft(s_pure)/len(s_pure)
ft_tones = fft(tones)/len(tones)

#plt.plot(20*log10(abs(ft_pure)))
plt.plot(freq, 20*log10(abs(ft_tones)))
plt.title('DTMF tones')
plt.ylabel('Magnitude (dB)')
plt.xlabel('Frequency (Hz)')
#plt.legend(['voice signal wave FFT'], loc='best')
plt.show()

## Ouvir tons DTMF

In [ ]:
# makes array c_contiguous in memory
tones = np.ascontiguousarray(tones, dtype=np.int16)

play_obj = sa.play_buffer(tones, num_channels, bytes_per_sample, sample_rate)
play_obj.wait_done()

## Adição dos tons ao sinal de voz

## Sinal de entrada + tons no tempo

In [ ]:
plt.plot(s)
plt.xlabel('Samples (n)')
plt.show()

## Visualização do sinal de entrada + tons na frequência 

In [ ]:
# gets fft frequency to plot properly
n = s.size
timestep = 1.0 / sample_rate
freq = np.fft.fftfreq(n, d=timestep)

#ft_pure = fft(s_pure)/len(s_pure)
ft_s = fft(s)/len(s)

#plt.plot(20*log10(abs(ft_pure)))
plt.plot(freq, 20*log10(abs(ft_s)))
plt.title('Voice signal with DTMF tones')
plt.ylabel('Magnitude (dB)')
plt.xlabel('Frequency (Hz)')
plt.show()

## Ouvir sinal de entrada com tons

In [ ]:
# makes array c_contiguous in memory
s = np.ascontiguousarray(s, dtype=np.int16)

play_obj = sa.play_buffer(s, num_channels, bytes_per_sample, sample_rate)
#play_obj.wait_done()

## Projeto do filtro anti-recobrimento

### 

### `firwin`

In [ ]:
# filter design FIR window algorithm
from scipy.signal import firwin
lpf1 = firwin(numtaps=40, cutoff=7000.0, window='hamming', nyq=nyq_rate) # cutoff is a fraction of Nyquist frequency

from scipy.signal import freqz
w, h = freqz(lpf1)

plt.plot((w/(2*pi))*sample_rate, 20*log10(abs(h)))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain (dB)')
plt.show()

## Escolher o filtro (adicionar diferentes opções, janelas, etc)

In [ ]:
lpf = lpf1
#lpf = lpf2

## Aplicar o filtro 
Utiliza a função `filtfilt` que resulta em fase linear (sem delay, ou delay uniforme).

In [ ]:
# filtering the signal with lfilter
from scipy.signal import filtfilt
s1 = filtfilt(lpf, 1, s)

## Sinal filtrado no tempo

In [ ]:
#plotting in time

plt.plot(s[2000:2100], '.-')
plt.plot(s1[2000:2100], '.-')
plt.xlabel('Samples (n)')
plt.title('Filtering result')
legend(('input signal', 'filtered signal'), loc='best')
plt.show()

In [ ]:
#plotting in time
#plt.plot(s_pure[:50])
#plt.plot(s[50000:50100])
#plt.plot(s1[50000:50100])
#plt.title('High frequency tone excerpt')
#legend(('input signal', 'filtered signal'), loc='best')
#plt.show()

In [ ]:
# plotting in frequency

# gets fft frequency to plot properly
n = s.size
timestep = 1.0 / sample_rate
freq = np.fft.fftfreq(n, d=timestep)

ft = fft(s)/len(s)
ft1 = fft(s1)/len(s1)

plt.plot(freq, 20*log10(abs(ft)))
plt.plot(freq, 20*log10(abs(ft1)))
plt.title('Voice signal before and after antialiasing filter')
plt.ylabel('Magnitude (dB)')
plt.xlabel('Frequency (Hz)')
legend(('input sig. FFT', 'filtered sig. FFT'), loc='best')
plt.show()

## Ouvir sinal de entrada filtrado

In [ ]:
# makes array c_contiguous in memory
s1 = np.ascontiguousarray(s1, dtype=np.int16)

play_obj = sa.play_buffer(s1, num_channels, bytes_per_sample, sample_rate)
play_obj.wait_done()

## Redução da taxa de amostragem

In [ ]:
sample_rate_down = 44100/6             # sampling frequency [Hz]
t_down = np.linspace(0, 1, sample_rate_down, endpoint=False)

nyq_rate_down = sample_rate_down / 2.0

In [ ]:
from scipy.signal import decimate
s_down = decimate(s1, 6, 150, 'fir', zero_phase=True)

In [ ]:
plt.plot(t[0:120], s1[0:120], '.-', t_down[0:20], s_down[0:20], 'o-')#, 0.01, s1[0], 'ro')
plt.xlabel('Time (s)')
print('Frequência de amostragem (Hz): ' + str(sample_rate_down))
print('Frequência de Nyquist    (Hz): ' + str(nyq_rate_down))
print('Bits por amostra             : ' + str(bytes_per_sample*8))

plt.legend(['orignal', 'downsampled'], loc='best')
plt.show()

In [ ]:
# plotting in frequency

n = s_down.size
timestep = 1.0 / sample_rate_down
freq = np.fft.fftfreq(n, d=timestep)

#ft1 = fft(s1)/len(s1) # obs.: already calculated previously in the code
ft2 = fft(s_down)/len(s_down)

#plt.plot(20*log10(abs(ft1)))
plt.plot(freq, 20*log10(abs(ft2)))
plt.title('Decimated signal')
plt.ylabel('Magnitude (dB)')
plt.xlabel('Frequency (Hz)')
plt.show()

## Ouvir sinal dizimado (somente por curiosidade, não aconteceria na prática) 

In [ ]:
# makes array c_contiguous in memory
#f = np.ascontiguousarray(f, dtype=np.int16)

#play_obj = sa.play_buffer(f, num_channels, bytes_per_sample, 8000) # 8 kHz just to test, as it is close to 7.35 kHz and is a standard audio freq
#play_obj.wait_done()

## Processamento do sinal (a fazer)

In [ ]:
plt.plot(s_down)
plt.show()

In [147]:
256/fs_down

0.034829931972789115

## Janelamento para detecção "sequencial" de tons

In [103]:
# generating test signals
SAMPLE_RATE = sample_rate_down
WINDOW_SIZE = 256 
#t = np.linspace(0, 1, SAMPLE_RATE)[:WINDOW_SIZE]

offset = 0 # offsets goertzel analysis (seconds)
offset_samples = int(offset * SAMPLE_RATE)

#loops = int(round(len(s_down)/WINDOW_SIZE)) # number of times to apply window and analyse spectrum
loops = 10

## No plotting, just find the frequency and tone

In [145]:
loops = 300

#fig, axes = plt.subplots(nrows=loops, ncols=2, sharex=False, sharey=False, figsize=(5,10))

threshold = 0.5e11
f_interesting = []


for i in range(loops):

    # parameters to shift the window
    start = (i*WINDOW_SIZE) + offset_samples
    end   = start + WINDOW_SIZE
    
    t = np.linspace(0, duration, duration*SAMPLE_RATE)#[start:end] # fixes t axis to correspond to part of the signal

    # apply window to parts of the signal
    sine_wave = s_down[start:end]#np.sin(2*np.pi*440*t) + np.sin(2*np.pi*1020*t)
    sine_wave = sine_wave * np.hamming(WINDOW_SIZE) 

    # applying Goertzel on those signals, and plotting results
    freqs, results = goertzel(sine_wave, SAMPLE_RATE, (690,950), (1200, 1640))#(100, 1000))#, 1000),  (1150, 1700))

    f = np.array(freqs)
    r = np.array(results)[:,2]
    print('---' + str(i) + '---')
    for j in range(len(freqs)):
        if r[j] > threshold:
            print(f[j])
            f_interesting.append(f[j])
    

### LOOP END

---0---
---1---
---2---
---3---
---4---
---5---
---6---
---7---
689.0625
717.7734375
1205.859375
---8---
689.0625
717.7734375
1205.859375
---9---
689.0625
717.7734375
1205.859375
---10---
689.0625
717.7734375
1205.859375
---11---
689.0625
717.7734375
1205.859375
---12---
689.0625
1320.703125
1349.4140625
---13---
689.0625
1320.703125
1349.4140625
---14---
689.0625
717.7734375
1320.703125
1349.4140625
---15---
689.0625
1320.703125
1349.4140625
---16---
689.0625
717.7734375
1320.703125
1349.4140625
---17---
689.0625
717.7734375
1320.703125
1349.4140625
---18---
689.0625
717.7734375
1464.2578125
1492.96875
---19---
689.0625
717.7734375
1464.2578125
1492.96875
---20---
689.0625
717.7734375
1464.2578125
1492.96875
---21---
689.0625
717.7734375
1464.2578125
1492.96875
---22---
689.0625
717.7734375
1464.2578125
1492.96875
---23---
689.0625
1464.2578125
1492.96875
---24---
775.1953125
1205.859375
---25---
775.1953125
1205.859375
---26---
775.1953125
1205.859375
---27---
775.1953125
1205.859375

In [138]:
f_interesting

[689.0625,
 717.7734375,
 1177.1484375,
 1205.859375,
 1234.5703125,
 689.0625,
 717.7734375,
 1177.1484375,
 1205.859375,
 1234.5703125,
 689.0625,
 717.7734375,
 1177.1484375,
 1205.859375,
 1234.5703125,
 689.0625,
 717.7734375,
 1177.1484375,
 1205.859375,
 1234.5703125,
 689.0625,
 717.7734375,
 1177.1484375,
 1205.859375,
 1234.5703125,
 689.0625,
 717.7734375,
 1177.1484375,
 1205.859375,
 1234.5703125,
 689.0625,
 717.7734375,
 1320.703125,
 1349.4140625,
 689.0625,
 717.7734375,
 1320.703125,
 1349.4140625,
 689.0625,
 717.7734375,
 1320.703125,
 1349.4140625,
 689.0625,
 717.7734375,
 1320.703125,
 1349.4140625,
 689.0625,
 717.7734375,
 1320.703125,
 1349.4140625,
 689.0625,
 717.7734375,
 1320.703125,
 1349.4140625,
 689.0625,
 717.7734375,
 1464.2578125,
 1492.96875,
 689.0625,
 717.7734375,
 1464.2578125,
 1492.96875,
 689.0625,
 717.7734375,
 1464.2578125,
 1492.96875,
 689.0625,
 717.7734375,
 1464.2578125,
 1492.96875,
 689.0625,
 717.7734375,
 1464.2578125,
 1492.9687

In [130]:
f = np.array(freqs)

In [131]:
np.array(results)[:,2]

array([  26489.55521087,   42337.09482503,   15562.1525439 ,
         33562.7007486 ,   13619.50460489,   55563.88877771,
        115761.82289073,   38241.10567836,   48510.72653747,
         21378.08598589,    2447.95489622,    4925.37167563,
          2424.59233132,    6892.40380972,   31908.88743923,
         54597.11701988,   15076.54958556,    1422.19758064,
           137.29052668,    3471.17554338,   12805.48564354,
         10939.06373108,    1910.61489789,    7620.55301453,
         13850.43773909,   15236.02665284,    3445.12408838])

## Aumento da taxa de amostragem

In [ ]:
from scipy.signal import resample

g = resample(s_down, len(s)) # upsamples to number of samples in original signal

plt.plot(g)
plt.xlabel('Samples (n)')
plt.show()

In [ ]:
n = g.size
timestep = 1.0 / sample_rate
freq = np.fft.fftfreq(n, d=timestep)

ft_g = fft(g)/len(g)

plt.plot(freq, 20*log10(abs(ft_g)))
plt.xlabel('Frequency (Hz)')
plt.title('Resulting voice signal')
plt.show()

## Ouvir sinal de saída (re)amostrado a 44,1 kHz

In [ ]:
# makes array c_contiguous in memory
g = np.ascontiguousarray(g, dtype=np.int16)

play_obj = sa.play_buffer(g, num_channels, bytes_per_sample, sample_rate)
play_obj.wait_done()

## Extra code

In [ ]:
# filter design FIR window algorithm
from scipy.signal import firwin
lpf1 = firwin(numtaps=40, cutoff=0.4, window='hamming') # cutoff is a fraction of Nyquist frequency

from scipy.signal import freqz
w, h = freqz(lpf1)

plt.plot(w/(2*pi), 20*log10(abs(h)))
plt.show()

### `firwin2`

In [ ]:
# filter design FIR window 2 algorithm
from scipy.signal import firwin2
lpf2 = firwin2(50, [0.0, 0.3, 0.45, 1.0], [1.0, 1.0, 0.0, 0.0], window='hamming') # cutoff is a fraction of Nyquist frequency
#lpf2 = firwin2(50, [0.0, 0.3, 0.45, 1.0], [1.0, 1.0, 0.0, 0.0], window='rectangular') # cutoff is a fraction of Nyquist frequency


from scipy.signal import freqz
w, h = freqz(lpf2)

plt.plot(w/(2*pi), 20*log10(abs(h)))
plt.show()